In [1]:
import numpy as np
from tqdm import tqdm_notebook
import itertools

In [2]:
N = 40
k = N-1
covmat = np.random.rand(N,N)
covmat[[range(0,N)],[range(0,N)]] = N
arr = list(range(0,N))
#covmat

In [3]:
#arr

In [4]:
covmat = 0.5 * (covmat+covmat)

In [5]:
from ipyparallel import Client
rc = Client(profile='pucv-cluster')
rc.purge_everything()

In [6]:
rc

In [7]:
#rc.ids

In [8]:
dview = rc[:] # use all engines

In [9]:
dview['covmat'] = covmat

In [10]:
dview.block = True
#dview.pull('covmat', targets=0)

In [11]:
"""N//"""

'N//'

In [12]:
sub_mat_n = N//2
hola = itertools.combinations(arr, sub_mat_n)

In [13]:
from math import factorial
n_el = N

n = factorial(n_el) / factorial(sub_mat_n) / factorial(n_el-sub_mat_n)
n_per_process = int(n)//59#len(rc.ids) 

print(n,n_per_process)

137846528820.0 2336381844


In [14]:
from itertools import islice, chain

def batch(iterable, size):
    sourceiter = iter(iterable)
    while True:
        batchiter = islice(sourceiter, size)
        yield chain([next(batchiter)], batchiter)

In [15]:
indexes = []
for i in range(len(rc.ids)):
    batchiter = islice(hola, int(round(n/len(rc.ids),0)))
    ind = chain([next(batchiter)], batchiter)
    indexes.append(ind)
    
#for el in hola2:
#    print(el)

In [16]:
#indexes

In [17]:
dview.scatter('index_mat',indexes)

In [18]:
import hashlib
import base64
m = hashlib.sha1()

def get_hash(string):
    m.update(string.encode())
    return base64.urlsafe_b64encode(m.digest())


In [19]:
ids_dict = {}
for i in range(len(rc)):
    ids_dict[i] = get_hash(str(i))
    rc[i]['hash_id'] = ids_dict[i]


In [20]:
with dview.sync_imports():
    import numpy
    import os
    import pickle

importing numpy on engine(s)
importing os on engine(s)
importing pickle on engine(s)


In [21]:
dview['hash_id'][:5]

[b'tlifxqsNyCzxIJnRwtQKuZToQQw=',
 b'3f4WM0XTOBk6wr3Bg_jp3P-QS0M=',
 b'xKLZm8KNI2CYoJUne36wcY1r4Gg=',
 b'xLXIa9V32j2T_qfInLphx4tI5Yk=',
 b'EZBKTot39iQuLSiHBQI62tAKkxA=']

In [22]:
hola = dview['hash_id'][:1]

In [23]:
hola[0].decode()

'tlifxqsNyCzxIJnRwtQKuZToQQw='

In [24]:
def det(N, level = 2):
    dets = numpy.zeros(N,dtype=numpy.float32)
    idx_mat = index_mat[0]
    i=0
    f = open("/home/srodriguez/res_mats/"+ hash_id.decode() + str(level) +".pkl","wb")
    print("creado f")
    for ind in idx_mat:
        idx = list(ind)
        matrix = covmat[idx][:,idx]
        dets[i] = numpy.linalg.det(matrix)
        i+=1
        #dets.append(numpy.linalg.det(matrix))
    print("Salvar matriz")
    pickle.dump(dets,f)
    f.close()
    #return dets

In [ ]:
ar = dview.apply_async(det,int(round(n/len(rc.ids),0)),sub_mat_n)
ar.wait_interactive()

   0/61 tasks finished after 4916 s

In [ ]:
res = ar.get()

In [ ]:
import pickle

In [ ]:
f = open("/home/srodriguez/res_mats/_fi8WBRTb2YBKIThRqiIekRwmlY=15.pkl","rb")
det_ejemplo = pickle.load(f)

In [ ]:
det_ejemplo[0]